# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-27-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-27-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-02-28 05:22:20,33.93911,67.709953,55707,2443,49288,3976.0,Afghanistan,143.101398,4.385445
1,NaN,NaN,NaN,Albania,2021-02-28 05:22:20,41.15330,20.168300,106215,1775,68969,35471.0,Albania,3690.840225,1.671139
2,NaN,NaN,NaN,Algeria,2021-02-28 05:22:20,28.03390,1.659600,112960,2979,77976,32005.0,Algeria,257.599346,2.637217
3,NaN,NaN,NaN,Andorra,2021-02-28 05:22:20,42.50630,1.521800,10849,110,10429,310.0,Andorra,14041.286482,1.013918
4,NaN,NaN,NaN,Angola,2021-02-28 05:22:20,-11.20270,17.873900,20782,506,19315,961.0,Angola,63.232004,2.434799


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55557,55575,55580,55604,55617,55646,55664,55680,55696,55707
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,96838,97909,99062,100246,101285,102306,103327,104313,105229,106215
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,111418,111600,111764,111917,112094,112279,112461,112622,112805,112960


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2430,2430,2430,2432,2433,2435,2436,2438,2442,2443
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1617,1636,1653,1666,1681,1696,1715,1736,1756,1775
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2950,2954,2958,2961,2964,2967,2970,2973,2977,2979


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,48798,48803,48820,48834,48895,48967,49086,49281,49285,49288
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,60675,61605,62533,63329,64318,65403,66309,67158,68007,68969
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,76640,76797,76940,77076,77225,77382,77537,77683,77842,77976


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6071,6079,6092,6117,6121,6143,6172,6203,6228,6248
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,19324,19361,19392,19433,19461,19554,19635,19670,19698,19714
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2057,2061,2067,2070,2074,2084,2095,2099,2106,2113


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,84,85,85,85,85,84,85,89,90,91
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,254,262,262,262,262,263,272,274,277,283
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,49,49,50,50,50,50,50,51,51,51


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-02-28 05:22:20,32.539527,-86.644082,6248,91,0,6157.0,"Autauga, Alabama, US",11183.303800,1.456466
689,1075.0,Lamar,Alabama,US,2021-02-28 05:22:20,33.779950,-88.096680,1325,33,0,1292.0,"Lamar, Alabama, US",9597.971749,2.490566
690,1077.0,Lauderdale,Alabama,US,2021-02-28 05:22:20,34.901719,-87.656247,8594,211,0,8383.0,"Lauderdale, Alabama, US",9267.866579,2.455201


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,28554465,511994,0
India,11096731,157051,10775169
Brazil,10517232,254221,9371448


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,28554465,511994,0,28042471,2021-02-28 05:22:20,37.936303,-91.379001
India,11096731,157051,10775169,164511,2021-02-28 05:22:20,23.088275,81.806127
Brazil,10517232,254221,9371448,891563,2021-02-28 05:22:20,-12.669522,-48.480493
Russia,4187166,84330,3756808,346028,2021-02-28 05:22:20,54.546312,62.120860
United Kingdom,4182772,122939,11602,4048231,2021-02-28 05:22:20,28.164647,-33.966498


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3563578,51953,0
Texas,2649363,43519,0
Florida,1903682,30734,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3563578,51953,0,3511625,2021-02-28 05:22:20,37.843962,-120.728594
Texas,2649363,43519,0,2605844,2021-02-28 05:22:20,31.660643,-98.653069
Florida,1903682,30734,0,1872948,2021-02-28 05:22:20,28.940755,-82.700744
New York,1636040,47498,0,1588542,2021-02-28 05:22:20,42.544151,-75.474183
Illinois,1185367,22710,0,1162657,2021-02-28 05:22:20,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1190894,21328,0
Arizona,Maricopa,509683,9107,0
Illinois,Cook,473944,9351,0
Florida,Miami-Dade,409216,5422,0
Texas,Harris,348848,5041,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1190894,21328,0,1169566,2021-02-28 05:22:20,34.308284,-118.228241,6037.0
Arizona,Maricopa,509683,9107,0,500576,2021-02-28 05:22:20,33.348359,-112.491815,4013.0
Illinois,Cook,473944,9351,0,464593,2021-02-28 05:22:20,41.841448,-87.816588,17031.0
Florida,Miami-Dade,409216,5422,0,403794,2021-02-28 05:22:20,25.611236,-80.551706,12086.0
Texas,Harris,348848,5041,0,343807,2021-02-28 05:22:20,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-25,55680,104313,112622,10799,20695,701,2093645,171227,28957,452767,...,4166727,55695,79773,1,137871,2421,179293,2255,77171,35994
2021-02-26,55696,105229,112805,10822,20759,701,2098728,171510,28965,454860,...,4175315,56542,79804,1,138295,2426,180848,2267,77639,36044
2021-02-27,55707,106215,112960,10849,20782,726,2104197,171793,28970,457317,...,4182772,57362,79836,1,138739,2432,181909,2269,78202,36058


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-25,2438,1736,2973,110,502,14,51795,3179,909,8493,...,122303,595,622,0,1334,35,2008,625,1059,1458
2021-02-26,2442,1756,2977,110,504,14,51887,3183,909,8515,...,122648,601,622,0,1338,35,2019,627,1066,1463
2021-02-27,2443,1775,2979,110,506,14,51946,3190,909,8538,...,122939,603,622,0,1341,35,2025,631,1081,1463


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-25,49281,67158,77683,10356,19238,271,1886732,162984,22900,425786,...,11504,48496,78290,1,129927,1804,164557,1434,70800,32455
2021-02-26,49285,68007,77842,10394,19307,271,1892834,163165,22902,427257,...,11579,49012,78333,1,130379,1839,165205,1434,72635,32539
2021-02-27,49288,68969,77976,10429,19315,291,1899087,163309,22903,428904,...,11602,49466,78373,1,130834,1844,166119,1435,73609,32590


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6071,6079,6092,6117,6121,6143,6172,6203,6228,6248
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1286,1291,1292,1294,1296,1306,1316,1317,1324,1325
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8508,8513,8515,8523,8525,8541,8570,8581,8588,8594
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2787,2790,2794,2795,2798,2806,2812,2813,2822,2826
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,14729,14766,14779,14798,14809,14827,14883,14900,14926,14950


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-25,491110,58151,0,812907,318122,3554629,424677,279159,85801,49,...,770940,2632666,369433,14840,2636,570982,337653,130813,615225,54202
2021-02-26,491849,58263,0,814528,318638,3559311,426198,279946,86098,49,...,772513,2643463,370084,14963,2636,572639,338822,131234,616018,54350
2021-02-27,492683,58263,0,815707,319195,3563578,427462,279946,86517,49,...,773887,2649363,370770,15098,2646,574314,339773,131580,616886,54350


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-02-25        6203   19670    2099  2442   6086    1163   1951   13175   
2021-02-26        6228   19698    2106  2445   6084    1163   1952   13202   
2021-02-27        6248   19714    2113  2449   6095    1167   1956   13232   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-25         3392     1764  ...    2606    587     3015      673   
2021-02-26         3399     1764  ...    2615    589     3018      675   
2021-02-27         3406     1768  ...    2615    589     3018      675   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-25           3731  3342  2048          0      880    622  
2021-02-26           3747  3348  2054          0      881    622  
2021-02-27           3747  3348  2054          0      881    622  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-25,9831,290,0,15814,5397,51395,5925,7614,1406,0,...,11321,43085,1890,203,25,7963,4942,2290,6994,671
2021-02-26,9869,290,0,15897,5407,51794,5940,7622,1418,0,...,11377,43376,1907,204,25,8197,4956,2291,6999,671
2021-02-27,9930,290,0,15967,5417,51953,5945,7622,1418,0,...,11393,43519,1929,204,25,8382,4956,2297,7014,671


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-02-25          89     274      51   60    125      36     65     285   
2021-02-26          90     277      51   60    125      36     65     285   
2021-02-27          91     283      51   60    127      36     66     286   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-25          108       38  ...      26     11       27        7   
2021-02-26          108       38  ...      26     11       27        7   
2021-02-27          110       38  ...      26     11       27        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-25             34     9    12          0       26      5  
2021-02-26             34     9    12          0       26      5  
2021-02-27             34     9    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-25,0.000287,0.009543,0.001432,0.002227,0.002665,0.085139,0.003948,0.001650,0.000345,0.005309,...,0.002411,0.014962,0.000301,0.0,0.003099,0.003731,0.008579,0.015308,0.008982,0.000945
2021-02-26,0.000287,0.008781,0.001625,0.002130,0.003093,0.000000,0.002428,0.001653,0.000276,0.004623,...,0.002061,0.015208,0.000389,0.0,0.003075,0.002065,0.008673,0.005322,0.006064,0.001389
2021-02-27,0.000198,0.009370,0.001374,0.002495,0.001108,0.035663,0.002606,0.001650,0.000173,0.005402,...,0.001786,0.014502,0.000401,0.0,0.003211,0.002473,0.005867,0.000882,0.007252,0.000388


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-25,0.000821,0.012245,0.001010,0.0,0.001996,0.0,0.002807,0.001260,0.0,0.002715,...,0.002656,0.006768,0.0,NaN,0.002254,0.0,0.004502,0.001603,0.007612,0.001374
2021-02-26,0.001641,0.011521,0.001345,0.0,0.003984,0.0,0.001776,0.001258,0.0,0.002590,...,0.002821,0.010084,0.0,NaN,0.002999,0.0,0.005478,0.003200,0.006610,0.003429
2021-02-27,0.000410,0.010820,0.000672,0.0,0.003968,0.0,0.001137,0.002199,0.0,0.002701,...,0.002373,0.003328,0.0,NaN,0.002242,0.0,0.002972,0.006380,0.014071,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-25,0.003973,0.012804,0.001883,0.003586,0.000884,0.163090,0.002212,0.002874,0.000131,0.003389,...,0.002615,0.009597,0.000486,0.0,0.003398,0.000000,0.004652,0.000698,0.014283,0.001388
2021-02-26,0.000081,0.012642,0.002047,0.003669,0.003587,0.000000,0.003234,0.001111,0.000087,0.003455,...,0.006519,0.010640,0.000549,0.0,0.003479,0.019401,0.003938,0.000000,0.025918,0.002588
2021-02-27,0.000061,0.014146,0.001721,0.003367,0.000414,0.073801,0.003304,0.000883,0.000044,0.003855,...,0.001986,0.009263,0.000511,0.0,0.003490,0.002719,0.005533,0.000697,0.013410,0.001567


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-25,0.001816,0.003122,NaN,0.001156,0.002287,0.001561,0.002642,0.003505,0.003450,0.0,...,0.002593,0.002977,0.002257,0.004875,0.004956,0.003579,0.003233,0.003306,0.001571,0.003964
2021-02-26,0.001505,0.001926,NaN,0.001994,0.001622,0.001317,0.003582,0.002819,0.003461,0.0,...,0.002040,0.004101,0.001762,0.008288,0.000000,0.002902,0.003462,0.003218,0.001289,0.002731
2021-02-27,0.001696,0.000000,NaN,0.001447,0.001748,0.001199,0.002966,0.000000,0.004867,0.0,...,0.001779,0.002232,0.001854,0.009022,0.003794,0.002925,0.002807,0.002637,0.001409,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-25      0.005023  0.001783  0.001909  0.002052  0.002306 -0.001717   
2021-02-26      0.004030  0.001423  0.003335  0.001229 -0.000329  0.000000   
2021-02-27      0.003211  0.000812  0.003324  0.001636  0.001808  0.003439   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-25      0.000000  0.006494 -0.000295  0.000567  ...  0.000768   
2021-02-26      0.000513  0.002049  0.002064  0.000000  ...  0.003454   
2021-02-27      0.002049  0.002272  0.002059  0.002268  ...  0.000000   

Province_State                                                              \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton    Uinta   
2021-02-25      0.003419  0.004330  0.001488   0.007834  0.001799  0.00540   
2021-02-26      0.003407  0.000995  0.002972   0.004288  0.001795  0.00293   
2021-02-27      0.000000  0.000000  0.000000   0.000000  0.000000  0.00000   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2021-02-25            NaN  0.000000    0.0  
2021-02-26            NaN  0.001136    0.0  
2021-02-27            NaN  0.000000    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-25,0.008929,0.0,NaN,0.007710,0.001856,0.008299,0.001352,0.002502,0.002853,NaN,...,0.004882,0.005930,0.005854,0.009950,0.0,0.019982,0.006107,0.002188,0.008362,0.0
2021-02-26,0.003865,0.0,NaN,0.005249,0.001853,0.007763,0.002532,0.001051,0.008535,NaN,...,0.004947,0.006754,0.008995,0.004926,0.0,0.029386,0.002833,0.000437,0.000715,0.0
2021-02-27,0.006181,0.0,NaN,0.004403,0.001849,0.003070,0.000842,0.000000,0.000000,NaN,...,0.001406,0.003297,0.011536,0.000000,0.0,0.022569,0.000000,0.002619,0.002143,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                               \
Admin2           Autauga   Baldwin Barbour      Bibb Blount   Bullock   
2021-02-25      0.047059  0.007353    0.02  0.016949  0.000  0.058824   
2021-02-26      0.011236  0.010949    0.00  0.000000  0.000  0.000000   
2021-02-27      0.011111  0.021661    0.00  0.000000  0.016  0.000000   

Province_State                                         ... Wyoming         \
Admin2            Butler   Calhoun  Chambers Cherokee  ...    Park Platte   
2021-02-25      0.000000  0.007067  0.048544      0.0  ...     0.0    0.0   
2021-02-26      0.000000  0.000000  0.000000      0.0  ...     0.0    0.0   
2021-02-27      0.015385  0.003509  0.018519      0.0  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2021-02-25          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-02-26          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-02-27          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2021-02-25        0.0  
2021-02-26        0.0  
2021-02-27        0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-25,0.000324,0.009904,0.001517,0.002507,0.002389,0.054641,0.003731,0.001454,0.000275,0.004694,...,0.002384,0.014599,0.000353,0.0,0.003218,0.003981,0.008103,0.012628,0.009410,0.001213
2021-02-26,0.000306,0.009343,0.001571,0.002319,0.002741,0.027321,0.003079,0.001553,0.000276,0.004658,...,0.002223,0.014903,0.000371,0.0,0.003147,0.003023,0.008388,0.008975,0.007737,0.001301
2021-02-27,0.000252,0.009356,0.001473,0.002407,0.001924,0.031492,0.002843,0.001602,0.000224,0.005030,...,0.002004,0.014703,0.000386,0.0,0.003179,0.002748,0.007128,0.004928,0.007494,0.000845


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-25,0.000671,0.011182,0.001018,0.002316,0.001839,0.012272,0.002783,0.001228,4.776221e-22,0.003220,...,0.003083,0.007301,4.914274e-08,NaN,0.003131,4.152211e-55,0.003904,0.002669,0.008719,0.003077
2021-02-26,0.001156,0.011351,0.001182,0.001158,0.002912,0.006136,0.002279,0.001243,2.388111e-22,0.002905,...,0.002952,0.008693,2.457137e-08,NaN,0.003065,2.076106e-55,0.004691,0.002934,0.007664,0.003253
2021-02-27,0.000783,0.011086,0.000927,0.000579,0.003440,0.003068,0.001708,0.001721,1.194055e-22,0.002803,...,0.002662,0.006010,1.228568e-08,NaN,0.002653,1.038053e-55,0.003831,0.004657,0.010868,0.001627


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-25,0.002892,0.013830,0.001934,0.003561,0.001361,0.08936,0.002483,0.001989,0.000172,0.003232,...,0.003292,0.010702,0.000519,0.0,0.003489,0.011582,0.004980,0.000527,0.011182,0.002302
2021-02-26,0.001486,0.013236,0.001990,0.003615,0.002474,0.04468,0.002859,0.001550,0.000130,0.003343,...,0.004906,0.010671,0.000534,0.0,0.003484,0.015492,0.004459,0.000263,0.018550,0.002445
2021-02-27,0.000774,0.013691,0.001856,0.003491,0.001444,0.05924,0.003081,0.001216,0.000087,0.003599,...,0.003446,0.009967,0.000522,0.0,0.003487,0.009105,0.004996,0.000480,0.015980,0.002006


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-25,0.002115,0.002876,NaN,0.001422,0.002226,0.001604,0.002584,0.004283,0.003200,2.374891e-105,...,0.002203,0.003024,0.002114,0.005489,0.006421,0.003390,0.002846,0.002644,0.001387,0.002511
2021-02-26,0.001810,0.002401,NaN,0.001708,0.001924,0.001460,0.003083,0.003551,0.003331,1.187445e-105,...,0.002122,0.003563,0.001938,0.006889,0.003211,0.003146,0.003154,0.002931,0.001338,0.002621
2021-02-27,0.001753,0.001201,NaN,0.001578,0.001836,0.001330,0.003024,0.001775,0.004099,5.937227e-106,...,0.001950,0.002897,0.001896,0.007956,0.003502,0.003035,0.002981,0.002784,0.001373,0.001310


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-25      0.004375  0.002745  0.003116  0.002418  0.002142  0.001192   
2021-02-26      0.004203  0.002084  0.003225  0.001823  0.000907  0.000596   
2021-02-27      0.003707  0.001448  0.003275  0.001730  0.001357  0.002018   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-25      0.000851  0.004746  0.001319  0.001446  ...  0.000737   
2021-02-26      0.000682  0.003398  0.001691  0.000723  ...  0.002096   
2021-02-27      0.001365  0.002835  0.001875  0.001495  ...  0.001048   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-25      0.003036  0.002660  0.001129   0.005203  0.001413  0.003368   
2021-02-26      0.003222  0.001827  0.002051   0.004745  0.001604  0.003149   
2021-02-27      0.001611  0.000914  0.001025   0.002373  0.000802  0.001574   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-25           -1.0  0.000308  0.000721  
2021-02-26           -1.0  0.000722  0.000360  
2021-02-27           -1.0  0.000361  0.000180  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-25,0.007738,0.000268,NaN,0.005977,0.001918,0.011089,0.001498,0.00255,0.006677,NaN,...,0.004738,0.005872,0.006003,0.008888,1.271566e-06,0.019881,0.005809,0.003178,0.005936,0.001722
2021-02-26,0.005802,0.000134,NaN,0.005613,0.001886,0.009426,0.002015,0.00180,0.007606,NaN,...,0.004842,0.006313,0.007499,0.006907,6.357829e-07,0.024633,0.004321,0.001807,0.003325,0.000861
2021-02-27,0.005991,0.000067,NaN,0.005008,0.001868,0.006248,0.001428,0.00090,0.003803,NaN,...,0.003124,0.004805,0.009518,0.003454,3.178914e-07,0.023601,0.002160,0.002213,0.002734,0.000431


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-25      0.025131  0.012978  0.010363  0.010771  0.000261  0.037051   
2021-02-26      0.018184  0.011964  0.005181  0.005385  0.000131  0.018525   
2021-02-27      0.014647  0.016812  0.002591  0.002693  0.008065  0.009263   

Province_State                                          ...  Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...     Park   
2021-02-25      0.001954  0.007097  0.030677  0.006757  ...  0.00004   
2021-02-26      0.000977  0.003548  0.015339  0.003378  ...  0.00002   
2021-02-27      0.008181  0.003529  0.016929  0.001689  ...  0.00001   

Province_State                                                                \
Admin2                Platte  Sheridan      Sublette    Sweetwater     Teton   
2021-02-25      4.697038e-11  0.000170  3.839522e-17  2.330898e-07  0.000123   
2021-02-26      2.348519e-11  0.000085  1.919761e-17  1.165449e-07  0.000061   
2021-02-27      1.174259e-11  0.000043  9.598804e-18  5.827246e-08  0.000031   

Province_State                                                       
Admin2                 Uinta Unassigned      Washakie        Weston  
2021-02-25      6.228666e-12  -0.501961  2.385559e-09  5.066395e-08  
2021-02-26      3.114333e-12  -0.501961  1.192780e-09  2.533197e-08  
2021-02-27      1.557166e-12  -0.501961  5.963898e-10  1.266599e-08  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210227,AK,55989,NaN,NaN,NaN,totalTestsViral,1679675,43.0,1277.0,...,NaN,0,0,22ba940bf484b87e67e142f5f1b88e1507f06ed4,0,0,0,0,0,NaN
1,20210227,AL,492683,106647.0,1900915.0,NaN,totalTestsPeopleViral,2286951,622.0,45428.0,...,NaN,61,0,3efe5c4666aeaac9a86de39874ee778a7c46250d,0,0,0,0,0,NaN
2,20210227,AR,319195,67169.0,2389866.0,NaN,totalTestsViral,2641892,475.0,14763.0,...,NaN,10,16,1792f462818197dfd2a0137a246bf00f56997af2,0,0,0,0,0,NaN
3,20210227,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,41630ed7561064a7f7fbd8c458f3299e0bb0d111,0,0,0,0,0,NaN
4,20210227,AZ,815707,55028.0,2986896.0,NaN,totalTestsViral,7623393,1317.0,57508.0,...,NaN,70,48,03c3e3eabe7cf771386bd5ca83ff05bbef94db0d,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210227,AK,55989.0,NaN,NaN,NaN,totalTestsViral,1679675.0,43.0,1277.0,...,NaN,0,0,22ba940bf484b87e67e142f5f1b88e1507f06ed4,0,0,0,0,0,NaN
1,20210227,AL,492683.0,106647.0,1900915.0,NaN,totalTestsPeopleViral,2286951.0,622.0,45428.0,...,NaN,61,0,3efe5c4666aeaac9a86de39874ee778a7c46250d,0,0,0,0,0,NaN
2,20210227,AR,319195.0,67169.0,2389866.0,NaN,totalTestsViral,2641892.0,475.0,14763.0,...,NaN,10,16,1792f462818197dfd2a0137a246bf00f56997af2,0,0,0,0,0,NaN
3,20210227,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,41630ed7561064a7f7fbd8c458f3299e0bb0d111,0,0,0,0,0,NaN
4,20210227,AZ,815707.0,55028.0,2986896.0,NaN,totalTestsViral,7623393.0,1317.0,57508.0,...,NaN,70,48,03c3e3eabe7cf771386bd5ca83ff05bbef94db0d,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-27,AK,55989,NaN,NaN,NaN,totalTestsViral,1679675,43.0,1277.0,NaN,...,NaN,0,0,22ba940bf484b87e67e142f5f1b88e1507f06ed4,0,0,0,0,0,NaN
2021-02-27,AL,492683,106647.0,1900915.0,NaN,totalTestsPeopleViral,2286951,622.0,45428.0,NaN,...,NaN,61,0,3efe5c4666aeaac9a86de39874ee778a7c46250d,0,0,0,0,0,NaN
2021-02-27,AR,319195,67169.0,2389866.0,NaN,totalTestsViral,2641892,475.0,14763.0,172.0,...,NaN,10,16,1792f462818197dfd2a0137a246bf00f56997af2,0,0,0,0,0,NaN
2021-02-27,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,41630ed7561064a7f7fbd8c458f3299e0bb0d111,0,0,0,0,0,NaN
2021-02-27,AZ,815707,55028.0,2986896.0,NaN,totalTestsViral,7623393,1317.0,57508.0,415.0,...,NaN,70,48,03c3e3eabe7cf771386bd5ca83ff05bbef94db0d,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-27,AK,55989.0,NaN,NaN,NaN,totalTestsViral,1679675.0,43.0,1277.0,NaN,...,NaN,0,0,22ba940bf484b87e67e142f5f1b88e1507f06ed4,0,0,0,0,0,NaN
2021-02-27,AL,492683.0,106647.0,1900915.0,NaN,totalTestsPeopleViral,2286951.0,622.0,45428.0,NaN,...,NaN,61,0,3efe5c4666aeaac9a86de39874ee778a7c46250d,0,0,0,0,0,NaN
2021-02-27,AR,319195.0,67169.0,2389866.0,NaN,totalTestsViral,2641892.0,475.0,14763.0,172.0,...,NaN,10,16,1792f462818197dfd2a0137a246bf00f56997af2,0,0,0,0,0,NaN
2021-02-27,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,41630ed7561064a7f7fbd8c458f3299e0bb0d111,0,0,0,0,0,NaN
2021-02-27,AZ,815707.0,55028.0,2986896.0,NaN,totalTestsViral,7623393.0,1317.0,57508.0,415.0,...,NaN,70,48,03c3e3eabe7cf771386bd5ca83ff05bbef94db0d,0,0,0,0,0,NaN
